In [16]:
using Pkg
Pkg.activate(".")

using MolecularGraph, MolecularGraph.Graphs

  Activating project at `~/.julia/dev/MOGMOG.jl`


In [14]:
# Tryptophan
supertypes(typeof(MolecularGraph.smilestomol("CC(C)(C)C")))

(SMILESMolGraph, SimpleMolGraph{Int64, SMILESAtom, SMILESBond}, AbstractMolGraph{Int64}, Graphs.AbstractGraph{Int64}, Any)

In [18]:
function dfs_with_backtrack(g::MolGraph, start::Int) # dfs, depth first search - trädsöknings metod 
    n = nv(g) # antalet atomer i grafen sparas i n 
    visited = falses(n) # en vektor visisted som berättar vilka som är besökta och inte. alla får false till en början 
    parent  = fill(0, n)  # en vektor av längd n och alla värden är 0 till att börja med. sparar alltså varje atoms förälder
    
    function _dfs(u) # en inre funktion som djupsöker. Tar in u som argument 
        visited[u] = true # markera u som besökt 
        for v in neighbors(g, u) # kolla efter u grannar 
            if !visited[v] # om grannen ej är besökt 
                parent[v] = u # då har vi nått v genom u, dvs u är föräldern till v 
                _dfs(v) # gör samma för v och alla grannar i grenen 
                # when we return here, we've finished exploring the entire v‐branch
                println("Finished branch at $v → jumping back to $u")
            end
        end
    end
    
    parent[start] = start   # root’s “parent” is itself
    _dfs(start) # börja med den första 
    return parent # returnera föräldar listan 
end

# Example usage:
mg = MolecularGraph.smilestomol("CC(C)(C)C") # Skapar en MolGraph från SMILES-strängen "CC(C)(C)C".
parents = dfs_with_backtrack(mg, 1) # funktionen returnerar en lista och har tagit in molekylen i mg och startindex 1. 


Finished branch at 3 → jumping back to 2
Finished branch at 4 → jumping back to 2
Finished branch at 5 → jumping back to 2
Finished branch at 2 → jumping back to 1


5-element Vector{Int64}:
 1
 1
 2
 2
 2

In [ ]:
# Ändringar och tillägg:
# - Infört två nya tokens ANCHOR_PUSH och ANCHOR_POP för att markera ankar-punkter vid DFS
# - Lagt till anchor_stack för att spara koordinater vid push och pop
# - Byggt atom_seq, coord_seq, atom_mask, coord_mask med ankar-tokens och dummy-koordinater (dvs utfyllnad för där push/pop är)
# - Uppdaterad loss-funktion som maskerar bort ankar-tokens i både atom- och koordinat-förlust
# - Behållt ursprunglig MOG-logpdf-stub och Flux-baserad atomtyp-förlust

using Graphs, MoleculeGraph, Flux, Distributions

# Definiera tokens för ankar-push och ankar-pop (lägg in i er atom_dict)
const ANCHOR_PUSH = 1001    # Token-ID för push
const ANCHOR_POP  = 1002    # Token-ID för pop

# DFS-traversering med explicit push/pop
function dfs_with_anchors(g::MolGraph, coords::Vector{SVector{3,Float32}}, start::Int)
    n = nv(g)                                   # antalet atomer i grafen sparas i n 
    visited      = falses(n)                    # en vektor visisted som berättar vilka som är besökta och inte. alla får false till en början 
    atom_seq     = Int[]                        # Sekvens av atom-ID:n + ankar-tokens. Ska ej ta in dessa saker som argument eftersom vi gör en ny av allt 
    coord_seq    = Vector{SVector{3,Float32}}() # Koordinatsekvens, inkluderar dummy för anchors
    atom_mask    = Bool[]                       # True för riktiga atomer, false för anchors
    coord_mask   = Bool[]                       # True för giltiga displacements
    anchor_stack = SVector{3,Float32}[]         # Sparar koordinater vid push

    function _traverse(u) # Början på den rekursiva hjälpfunktionen _traverse, som tar den aktuella noden/atomen u som argument.
        visited[u] = true # Markera nod u som besökt i vektorn visited, så vi inte återvänder hit igen.
        # Lägg till aktuell atom
        push!(atom_seq, u) 
        push!(coord_seq, coords[u])
        push!(atom_mask, true)
        push!(coord_mask, true)

        # Hitta oexplorerade grannar
        nbrs = [v for v in neighbors(g, u) if !visited[v]] # inte besökta grannar sparas i en lista 
        for v in nbrs # besök de ej besökta 
            if length(nbrs) >= 2 # Gör push om flera grenar. Om det skulle varit 1 finns det bara en väg framåt och är ej en förgrening
                push!(anchor_stack, coords[u]) # Spara aktuell position på stacken
                
                # Lägg till push-token med dummy-/ankar-koordinat för alla dessa eftersom detta sker om det är en förgrening och då ska vi ha push 
                push!(atom_seq, ANCHOR_PUSH)
                push!(coord_seq, coords[u])       # alternativt zeros(SVector{3,Float32})
                push!(atom_mask, false)           # exkludera i atomförlust
                push!(coord_mask, false)          # exkludera i koordinatförlust
            end

            _traverse(v) # Rekursivt gå in i grenen

            # Efter att grenen är utforskad: pop
            if length(nbrs) >= 2
                anchor_pos = pop!(anchor_stack) # Hämta sparad position

                # Lägg till pop-token med återställd ankar-koordinat
                push!(atom_seq, ANCHOR_POP)
                push!(coord_seq, anchor_pos)
                push!(atom_mask, false)
                push!(coord_mask, false)
            end
        end
    end

    _traverse(start) 
    return atom_seq, coord_seq, atom_mask, coord_mask
end

# Uppdaterad loss-funktion som ignorerar ankar-tokens
function loss(model, atom_seq, coord_seq, atom_mask, coord_mask)
    # Bygg indata för modellen
    atom_ids = Flux.onehotbatch(atom_seq, 1:model.vocab_size)  # gör en matris av varje atom. Tar atomtoken (siffran för att beskriva atomen) och beskriver med en one hot vektor. Vi måste skriva om de diskreta token på detta sätt för modellen
    pos      = reshape(hcat(coord_seq...), 3, 1, length(coord_seq))  # Ändrar lite format på koordinaterna [3, B=1, T]

    μ, σ, logw, logits = model(pos, atom_ids) # skicka in atom id och pos och returnera de värden som behövs för att beräkna förlust 

    disp = pos[:, :, 2:end] .- pos[:, :, 1:end-1] # Beräkna förskjutningar mellan på varandra följande tokens. Detta är då inte strikt atom 5-4 utan följer den ordning vi skapat. Dvs en gren i taget. 

    # Maskera bort displacements vid push/pop
    coord_mask_arr = reshape(coord_mask, 1, 1, length(coord_mask)) # reshapea
    disp .= disp .* coord_mask_arr[:, :, 2:end] # vektormultiplikation så vi får 0 för de falska och inte ska räknas med sedan 

    # Koordinat-förlust via MOG
    logp_xyz = logpdf_MOG(disp, μ, σ, logw) # Log-sannolikhet för dessa displacement enligt modellens Gauss-komponenter
    loss_xyz = -sum(logp_xyz .* coord_mask_arr[:, :, 2:end]) / sum(coord_mask) # maska bort false och ta negativ medelvärde

    # Atomtyp-förlust, ignorera anchors
    target_atoms   = atom_seq[2:end]
    atom_onehot    = Flux.onehotbatch(target_atoms, 1:size(logits,1))
    atom_mask_arr  = reshape(atom_mask, 1, length(atom_mask))

    masked_mean(p) = sum(p .* atom_mask_arr[:, 2:end]) / sum(atom_mask) # maska bort false och ta negativ medelvärde
    loss_type      = Flux.logitcrossentropy(dropdims(logits, dims=2), atom_onehot; agg=masked_mean)

    return loss_xyz + loss_type # slå ihop loss 
end

# Stub för log-sannolikhet av Mixture of Gaussians
typealias_VEC SVector{3,Float32}
function logpdf_MOG(disp, μ, σ, logw)
    return sum(logpdf.(MvNormal.(μ, σ), eachslice(disp, dims=3))) .+ logw
end